## REDSEA python version 0.0.1

Edited from the original REDSEA python version 0.0.1 file in this folder (see that for details).

Edited by Ben Caiello, with the intent of allowing it to work with current DeepCell segmentation masks / the steinbock pipeline. The algorithm is not identical, as that is not possible (I think) given the differenes in the masks of old DeepCell and new DeepCell (+ / - zero boundaries changes the effective pixel width of the boundary measurement step).

Dataset I had been using for CD3 / CD20 compensation: https://zenodo.org/records/8023452

Passed through steinbock to derive .tiffs and masks, then fed into this script

The directory structure required for the script as-written is the one naturally produced by steinbock:

A master directory with two folders: \img & \masks - each containing .tiffs with the original images and the DeepCell generated masks, repectively, with matching file names - and 1 .csv file (panel.csv). These are all naturally produced by steinbock when it is run.

The script has not been thoroughly tested, but should produce outputs and seems to be doing what it is supposed to with the limited testing so far.


Some (potential and certain) differences in the algorithm to note:
    1. Since the masks format is different with DeepCell now (no cell-cell padding with 0's), the algorithm is made to find border px by looking for >1 segmentation label (indicating >1 cell or cell + background boundary)
    2. Also, because of the lack of padding, the effective distances and sizes of the diamond / square boder px identification step is altered

In [18]:
# Package Imports
import PIL
from PIL import Image, ImageSequence, ImageOps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage 
import skimage.measure
import skimage.morphology
import glob
from scipy.io import loadmat
import time

import tifffile
import os

In [19]:
######## Edit these for your run!
# file locations
mainPath = 'C:\\Users\\caiello\\Desktop\\RedSEA practice\\practice' # main folder must contain \img and \masks folders and a panel.csv file (matching steinbock output)

# for multiple images at once:
file_list = []
for i in os.listdir(mainPath + "\\img"):
    i = i.replace(".tiff","")
    file_list.append(i)

# or set file_name manually
file_single = 'RNANeg_Tonsil_003'  #do not include the .tiff file type!

#  select which channel to normalize
normChannels = [13,18]    # I use numbers here, can switch to names as in your panel file -- if you set "channel_names_numbers = False" in the the file_reader call in the next cell 

# parameters for compensation (change as desired)
REDSEAChecker = 1 # 1 means subtract+ reinforce
elementShape = 1 # star, 1 == square size
elementSize = 1 # star or square extension size


# output path
pathResults = mainPath + '/intensities' # output location, set as intensities here to match steinbock output
try:
    os.listdir(pathResults)
except:
    os.mkdir(pathResults)

alt_path = "\\an\\alternate\folder\\in\\which\\to\\put\\your\\non-scaled\\and\\non-compensated\\data" ## use if you want the non-compensated / non-scaled data, but don't want them crowding the main steinbock intensities folder
######### if you use alt_path, manually make the directory in file explorer!!!

In [95]:
## Functions (normally, minimize this cell)
# helper function 1
def ismember(a, b):
    bind = {}
    for i, elt in enumerate(b):
        if elt not in bind:
            bind[elt] = i
    return [bind.get(itm, None) for itm in a]  # None can be replaced by any other "not in b" value

# helper function 2

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()
        
def file_reader(mainPath,file_name,channel_names_numbers = True):
    massDS_path = mainPath + '\\panel.csv' # csv file location, need
    pathTiff = mainPath + '\\img\\' + file_name + '.tiff' #  tiff location, links to a single .tiff
    pathMat = mainPath + '\\masks\\' + file_name + '.tiff' # corresponding .tiff's mask
        
    ####### Read in of files:
    ### read in panel.csv
    massDS = pd.read_csv(massDS_path) # read the mass csv
    if channel_names_numbers == True:
        clusterChannels = massDS[massDS['keep'] == 1].reset_index().index.values # only get the label column minus 1 is for zero-indexing (matched channel numbers with Napari)
                                                # remove -1 if you want 1-indexing of the channel names
    else:
        clusterChannels = massDS['name']  # if you want your channel names to be want is used
    channelNum = len(clusterChannels) # how many channels
    #print(massDS.head())
    
    #### this part reads in multichannel tiff file
    # read in the image and transform into a 'countsNoNoise' matrix
    array_list=[]
    for channel in clusterChannels:
        t=tifffile.imread(pathTiff)[channel]
        array_list.append(t)
    countsNoNoise=np.stack(array_list,axis=2) # count matrices in the image
    
    ###### Read in segmentation .tiff
        # Define the boundary region
        #### these code is just entire translation of redsea matlab v1.0
    Segmentation = tifffile.imread(pathMat).astype('int')
    cellNum = np.max(Segmentation) # how many labels
    stats = skimage.measure.regionprops(Segmentation+1) # get the regional props for all the labels

    ### make empty container matrices
    data = np.zeros((cellNum + 1,channelNum))
    dataScaleSize = np.zeros((cellNum + 1,channelNum))
    cellSizes = np.zeros((cellNum + 1,1))
    
    # this part extract counts data from the whole cell regions, for each individual cells etc
    
    for i in range(cellNum + 1): # for each cell (label)
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i,0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes

    return clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize

def cell_cell_matrix(Segmentation, cellNum, REDSEAChecker = REDSEAChecker):
    # this block is for computing cell cell matrix
    [rowNum, colNum] = Segmentation.shape
    cellPairMap = np.zeros(((cellNum + 1),(cellNum + 1))) # cell-cell shared perimeter matrix container
    
    # start looping the mask and produce the cell-cell contact matrix
    for i in range(rowNum):
        if i == 0:   # these conditional statements account for pixels on the edge of the image by shrinking the 3x3 box to smaller dimensions
            a = 0
            c = 2
        elif i == (rowNum - 1):
            a = 1
            c = 1
        else:
            a = 1
            c = 2
        for j in range(colNum):
            if j == 0:
                b = 0
                d = 2
            elif j == (colNum - 1):
                b = 1
                d = 1
            else:
                b = 1
                d = 2
            tempMatrix = Segmentation[i-a:i+c,j-b:j+d] # the 3x3 window, centered on the point i,j
            #print(tempMatrix)
            tempFactors = np.unique(tempMatrix).astype('int') #unique
            #print(tempFactors)
            centerpoint_value = Segmentation[i,j]
            #print(centerpoint_value)
            for k in tempFactors:
                if k != centerpoint_value: # only add to the cellPairMap for the centerpoint pixel -- this prevents multiplicate counting
                    #print("trigger")
                    cellPairMap[centerpoint_value,k] = cellPairMap[centerpoint_value,k] + 1  
        
    # converting the cell cell maps to fraction of cell - cell boundary (not of total cell boundary [!?] -- as in boundary with empty space not counted)
    cellPairNorm = np.zeros(((cellNum+1),(cellNum+1)))
    for i in np.arange(0,len(cellPairMap)):
        if np.sum(cellPairMap[i]) > 0:
            cellPairNorm[i] = - (cellPairMap[i] / np.sum(cellPairMap[i]))
        #else:
            #print("this shouldn't happen")
    cellPairNorm = cellPairNorm[1:,1:]
    cellPairNorm = cellPairNorm + REDSEAChecker*np.identity(cellNum ) 
    #display(pd.DataFrame(cellPairMap))
    print(pd.DataFrame(cellPairMap).sum())
    return cellPairNorm, rowNum, colNum

def Cell_Edge_Intensities(Segmentation, cellNum, channelNum, countsNoNoise, rowNum, colNum, elementShape = 2, elementSize = 2):
    MIBIdataNearEdge1 = np.zeros((cellNum+1,channelNum))

    ##### A List of Items
    items = list(range(cellNum + 1))
    l = len(items)
    printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50) # progress bar
    #####
    
    ######pre-calculated shape
    if elementShape==1: # square
        shape=skimage.morphology.square((2*elementSize) + 1)
    elif elementShape==2: # diamond
        shape=skimage.morphology.diamond(elementSize) # create diamond shapte based on elementSize
    else:
        print("Error elementShape Value not recognized.")
    ############
    
    
    for i in range(cellNum + 1) :
        if i == 0:
            continue
        [tempRow,tempCol] = np.asarray(Segmentation==i).nonzero()
        [rowNum, colNum] = Segmentation.shape
        # sequence in row not col, should not affect the code
        for j in range(len(tempRow)):
            label_in_shape=[] # empty list in case
            if (tempRow[j] - elementSize) <= -1:
                a = 0 + tempRow[j]
                c = 1 + elementSize
            elif (tempRow[j] + elementSize) > rowNum - 1:
                a = 0 + elementSize
                c = 1 + rowNum - tempCol[j]
            else:
                a = 0 + elementSize
                c = 1 + elementSize
            if (tempCol[j] - elementSize) <= -1:
                b = 0 + tempCol[j]
                d = 1 + elementSize
            elif (tempCol[j] + elementSize) > colNum - 1:
                b = 0 + elementSize
                d = 1 + colNum - tempCol[j]
            else:
                b = 0 + elementSize
                d = 1 + elementSize
            tempMatrix = Segmentation[tempRow[j]-a:tempRow[j]+c,tempCol[j]-b:tempCol[j]+d] # the 3x3 window, centered on the point i,j
            [dim1, dim2] = tempMatrix.shape
            reducedShape = shape[:dim1,:dim2].astype('bool')
            reducedMatrix = tempMatrix[reducedShape]   # the reduced shape /  matrix lines are there to accommodate the diamond shape
            tempFactors = np.unique(reducedMatrix).astype('int')
            if len(tempFactors) > 1:
                MIBIdataNearEdge1[i,:] = MIBIdataNearEdge1[i,:] + countsNoNoise[tempRow[j],tempCol[j],:]
            # Update Progress Bar
            if ((i % 500) == 0) or (i == (cellNum - 1)):
                printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
            return MIBIdataNearEdge1
            
def CalculateREDSEA(MIBIdataNearEdge1, cellPairNorm, data, channelNormIdentity, cellNum, dataScaleSize, cellSizes):    
    MIBIdataNorm2 = np.transpose(np.dot(np.transpose(MIBIdataNearEdge1[1:,:]),cellPairNorm))
    #this is boundary signal subtracted by cell neighbor boundary
    MIBIdataNorm2 = MIBIdataNorm2 + data[1:,:] # reinforce onto the whole cell signal (original signal)
    
    MIBIdataNorm2[MIBIdataNorm2<0] = 0 # clear out the negative ones
    # flip the channelNormIdentity for calculation
    rev_channelNormIdentity=np.ones_like(channelNormIdentity)-channelNormIdentity
    # composite the normalized channels with non-normalized channels
    # MIBIdataNorm2 is the matrix to return
    MIBIdataNorm2 = data[1:,:] * np.transpose(np.tile(rev_channelNormIdentity,(1,cellNum))) + MIBIdataNorm2 * np.transpose(np.tile(channelNormIdentity,(1,cellNum)))
    
    # the function should return 4 variables
    dataCells = data[1:,:]
    dataScaleSizeCells = dataScaleSize[1:,:]
    dataCompenCells = MIBIdataNorm2
    dataCompenScaleSizeCells = MIBIdataNorm2 / cellSizes[1:,:]

    return dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells

def data_matrix_with_label_and_cellsizes(data_in, cellNum, cellSizes, clusterChannels):
    ### in a separate funcito because this data is not likely to be wanted. still it can be accessed:
    labelVec = [i for i in np.arange(1,cellNum + 1,1)]
    cellSizesVec_flat = [item for sublist in cellSizes[1:,:] for item in sublist] # flat the list
    dataL = pd.DataFrame({'Object':labelVec, 'cell_size':cellSizesVec_flat})
    dataCells_df=pd.DataFrame(data_in)
    dataCells_df.columns = clusterChannels
    data_out = pd.concat((dataL,dataCells_df),axis=1)
    return data_out

def Outputter(data_in, file_name, pathResults = pathResults):
    data_in.to_csv(pathResults + '\\' + file_name + ".csv", index = False)

def RunRedSEA(mainPath, file, channel_names_numbers = True, pathResults = pathResults, alt_path = pathResults, normChannels = normChannels, print_options = "only_compensated_and_scaled"):
    if print_options not in ['only_compensated_and_scaled','only_scaled','all']:
        raise("Error: print_options not one of the following: 'only_compensated_and_scaled', 'only_scaled', or 'all'!") 
    if type(file) == list:
        for ii in file:
            file_name = ii
            print("Starting RedSEA compensation of file: " + file_name)
            clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize = file_reader(mainPath, ii, channel_names_numbers = channel_names_numbers)
            normChannelsInds = ismember(normChannels,clusterChannels)
            channelNormIdentity = np.zeros((len(clusterChannels),1))
            # make a flag for compensation
            for i in range(len(normChannelsInds)):
                    channelNormIdentity[normChannelsInds[i]] = 1 
            # print("cellNum for " + file_name + " " + str(cellNum))
            cellPairNorm, rowNum, colNum = cell_cell_matrix(Segmentation = Segmentation, REDSEAChecker = REDSEAChecker, cellNum = cellNum)
            MIBIdataNearEdge1 = Cell_Edge_Intensities(Segmentation, countsNoNoise = countsNoNoise, cellNum = cellNum, channelNum = channelNum, rowNum = rowNum, colNum = colNum, elementShape = 2, elementSize = 2)
            dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells = CalculateREDSEA(MIBIdataNearEdge1 = MIBIdataNearEdge1, cellPairNorm = cellPairNorm, data = data, channelNormIdentity = channelNormIdentity, cellNum = cellNum, dataScaleSize = dataScaleSize, cellSizes = cellSizes)
            dataL_full = data_matrix_with_label_and_cellsizes(dataCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            dataScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            dataCompenL_full = data_matrix_with_label_and_cellsizes(dataCompenCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            dataCompenScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataCompenScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            if print_options == "only_compensated_and_scaled":
                Outputter(dataCompenScaleSizeL_full, file_name = file_name)
            elif print_options == "only_scaled":
                Outputter(dataCompenScaleSizeL_full, file_name = file_name)
                Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA.csv", pathResults = alt_path)
            elif print_options == "all":
                Outputter(dataL_full, file_name = file_name + "_pre_REDSEA_unscaled.csv")
                Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA_scaled.csv")
                Outputter(dataCompenL_full, file_name = file_name + "_post_REDSEA_unscaled.csv")
                Outputter(dataCompenScaleSizeL_full, file_name = file_name)
    elif type(file) == str:
        file_name = file
        print("Starting RedSEA compensation of file: " + file_name)
        clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize = file_reader(mainPath, file_name, channel_names_numbers = channel_names_numbers)
        normChannelsInds = ismember(normChannels,clusterChannels)
        channelNormIdentity = np.zeros((len(clusterChannels),1))
        # make a flag for compensation
        for i in range(len(normChannelsInds)):
                channelNormIdentity[normChannelsInds[i]] = 1 
        # print("cellNum for " + file_name + " " + str(cellNum))
        cellPairNorm, rowNum, colNum = cell_cell_matrix(Segmentation = Segmentation, REDSEAChecker = REDSEAChecker, cellNum = cellNum)
        MIBIdataNearEdge1 = Cell_Edge_Intensities(Segmentation, countsNoNoise = countsNoNoise, cellNum = cellNum, channelNum = channelNum, rowNum = rowNum, colNum = colNum, elementShape = 2, elementSize = 2)
        dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells = CalculateREDSEA(MIBIdataNearEdge1 = MIBIdataNearEdge1, cellPairNorm = cellPairNorm, data = data, channelNormIdentity = channelNormIdentity, cellNum = cellNum, dataScaleSize = dataScaleSize, cellSizes = cellSizes)
        dataL_full = data_matrix_with_label_and_cellsizes(dataCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        dataScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        dataCompenL_full = data_matrix_with_label_and_cellsizes(dataCompenCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        dataCompenScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataCompenScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        if print_options == "only_compensated_and_scaled":
            Outputter(dataCompenScaleSizeL_full, file_name = file_name)
        elif print_options == "only_scaled":
            Outputter(dataCompenScaleSizeL_full, file_name = file_name)
            Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA.csv", pathResults = alt_path)
        elif print_options == "all":
            Outputter(dataL_full, file_name = file_name + "_pre_REDSEA_unscaled.csv")
            Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA_scaled.csv")
            Outputter(dataCompenL_full, file_name = file_name + "_post_REDSEA_unscaled.csv")
            Outputter(dataCompenScaleSizeL_full, file_name = file_name)
    return dataL_full, dataScaleSizeL_full, dataCompenL_full, dataCompenScaleSizeL_full # these are the files of only the last file, if running multiple files at once


In [96]:
# Whole folder run (this one had 4 files in it):
dataL_full, dataScaleSizeL_full, dataCompenL_full, dataCompenScaleSizeL_full = RunRedSEA(mainPath = mainPath, file = file_list, channel_names_numbers = True, normChannels = normChannels, print_options = "only_scaled")

Starting RedSEA compensation of file: RNANeg_Tonsil_001
0       107225.0
1            9.0
2           24.0
3           20.0
4           22.0
          ...   
7713        13.0
7714        13.0
7715        11.0
7716        13.0
7717         9.0
Length: 7718, dtype: float64
Starting RedSEA compensation of file: RNANeg_Tonsil_002------| 0.0% Complete
0        55651.0
1           11.0
2           14.0
3           16.0
4           12.0
          ...   
20905       44.0
20906       13.0
20907        9.0
20908       16.0
20909        9.0
Length: 20910, dtype: float64
Starting RedSEA compensation of file: RNANeg_Tonsil_003------| 0.0% Complete


KeyboardInterrupt: 

In [13]:
# pre-comp data from the last file in the run, scaled by cell size
dataScaleSizeL_full.head()

,Object,cell_size,0,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,24,25
0,1,16.0,0.062500,4.926793,0.788452,0.661649,0.652807,0.269625,0.062500,0.849422,...,0.345111,20.375765,8.192933,0.062500,0.156391,4.087760,6.344390,2.887861,21.861406,1.737484
1,2,14.0,0.159876,3.469197,2.454025,1.094108,0.722595,0.739134,0.179178,1.805447,...,0.142857,2.221802,4.049692,0.232816,0.278882,5.062777,9.115035,4.463692,23.167364,1.955778
2,3,25.0,0.000000,3.255748,1.553636,1.000429,0.605723,0.510202,0.094054,1.764787,...,0.151094,2.385300,9.332050,0.307272,0.224216,6.079697,13.259372,1.697419,13.507796,1.804346
3,4,30.0,0.112569,1.680611,1.273399,1.319794,0.470234,0.688838,0.289421,1.556288,...,0.418586,27.638899,2.014158,0.153056,0.243548,5.158520,9.989205,1.534308,10.946206,1.715446
4,5,22.0,0.000000,0.658532,1.813758,0.810043,0.331333,0.707729,0.136364,1.580072,...,0.394834,4.231170,1.833733,0.121115,0.110704,5.336914,9.130732,2.396150,35.118828,1.554082


In [14]:
# post-comp data from the last file in the run, scaled by cell size
dataCompenScaleSizeL_full.head()

,Object,cell_size,0,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,24,25
0,1,16.0,0.062500,4.926793,0.788452,0.661649,0.652807,0.269625,0.062500,0.849422,...,0.345111,20.375765,5.623331,0.062500,0.156391,4.087760,6.344390,2.887861,21.861406,1.737484
1,2,14.0,0.159876,3.469197,2.454025,1.094108,0.722595,0.739134,0.179178,1.805447,...,0.142857,2.221803,1.850488,0.232816,0.278882,5.062777,9.115035,4.463692,23.167365,1.955779
2,3,25.0,0.000000,3.255748,1.553636,1.000429,0.605723,0.510202,0.094054,1.764787,...,0.151094,2.385300,8.985885,0.307272,0.224216,6.079697,13.259371,1.697419,13.507797,1.804346
3,4,30.0,0.112569,1.680611,1.273399,1.319794,0.470234,0.688838,0.289421,1.556288,...,0.418586,27.638900,0.000000,0.153056,0.243548,5.158520,9.989205,1.534308,10.946206,1.715446
4,5,22.0,0.000000,0.658532,1.813759,0.810043,0.331333,0.707729,0.136364,1.580072,...,0.394834,4.231170,1.869282,0.121115,0.110704,5.336914,9.130732,2.396150,35.118827,1.554082


In [12]:
# single file run:

dataL_full, dataScaleSizeL_full, dataCompenL_full, dataCompenScaleSizeL_full = RunRedSEA(mainPath = mainPath, file = file_single, channel_names_numbers = True, normChannels = normChannels, print_options = "only_compensated_and_scaled")

Starting RedSEA compensation of file: RNANeg_Tonsil_003
0        149224.0
1            16.0
2            15.0
3            18.0
4            19.0
           ...   
13750        18.0
13751        10.0
13752        24.0
13753        13.0
13754        11.0
Length: 13755, dtype: float64
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [ ]:
### For testing / debugging:

clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize = file_reader(mainPath,file_single,channel_names_numbers = True)


In [ ]:
##### old stuff

def OLD_non_functional_Cell_Edge_Intensities(Segmentation, cellNum, channelNum, countsNoNoise, rowNum, colNum, elementShape = 2, elementSize = 2):    
    # now starts the calculation of signals from pixels along the boudnary of cells
    MIBIdataNearEdge1 = np.zeros((cellNum+1,channelNum))
    newLmod_border = np.pad(Segmentation, pad_width=elementSize, mode='constant', constant_values=0)
    # start the boundary region selection and count extraction
    
    ##### A List of Items
    items = list(range(cellNum))
    l = len(items)
    printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50) # progress bar
    #####
    
    ######pre-calculated shape
    if elementShape==1: # square
        square=skimage.morphology.square((2*elementSize) + 1)
        square_loc=np.where(square==1)
    elif elementShape==2: # diamond
        diam=skimage.morphology.diamond(elementSize) # create diamond shapte based on elementSize
        diam_loc=np.where(diam==1)
    else:
        print("Error elementShape Value not recognized.")
    ############
    
    for i in range(cellNum):
        if i = 0:
            continue
        [tempRow,tempCol] = np.where(Segmentation==i)
        # sequence in row not col, should not affect the code
        for j in range(len(tempRow)):
            label_in_shape=[] # empty list in case
            # make sure not expand outside
            #if (elementSize-1<tempRow[j]) and (tempRow[j]<rowNum-elementSize-2) and (elementSize-1<tempCol[j]) and (tempCol[j]<colNum-elementSize-2):
            ini_point = [tempRow[j]-elementSize,tempCol[j]-elementSize] # corrected top-left point
            if elementShape==1: # square
                square_loc_ini_x=[item + ini_point[0] for item in square_loc[0]]
                square_loc_ini_y=[item + ini_point[1] for item in square_loc[1]]
                
                label_in_shape=[Segmentation[square_loc_ini_x[k],square_loc_ini_y[k]] for k in range(len(square_loc_ini_x))]
                
            elif elementShape==2: # diamond
                diam_loc_ini_x=[item + ini_point[0] for item in diam_loc[0]]
                diam_loc_ini_y=[item + ini_point[1] for item in diam_loc[1]]
                # finish add to ini point
            
                label_in_shape=[Segmentation[diam_loc_ini_x[k],diam_loc_ini_y[k]] for k in range(len(diam_loc_ini_x))]
            is_border_px = len(np.unique(label_in_shape))
            
            if is_border_px > 1:
                MIBIdataNearEdge1[i,:] = MIBIdataNearEdge1[i,:] + countsNoNoise[tempRow[j],tempCol[j],:]
        
        # Update Progress Bar
        printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
    return MIBIdataNearEdge1